In [1]:
# retrieve plasmid sequence from GenBank
import os
from pydna.genbank import Genbank
from Bio import SeqIO
from Bio.Restriction import AllEnzymes
from pydna.parsers import parse_snapgene
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

In [2]:
AllEnzymes.get("EcoRI")

EcoRI

In [3]:
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design

In [4]:
def load_from_genebank(email, sequence):
    gb = Genbank(users_email=email)  
    seq = gb.nucleotide(sequence)   
    return seq

In [14]:
email = ""

In [6]:
# Some gene sequence
seqs_id = ["M61151.1", "X51424.1", "PV006683.1", "PV006682.1", "OR455449.1"]
# Create a Dseq object
seqs = []
for seq in seqs_id:
    seqs.append(load_from_genebank(email, seq))


In [7]:
# Define primer length (typical range 18-22 nt)
primer_length = 20

In [8]:
# Use Primer3 for more sophisticated design (optional):
primers = list(map(lambda seq: primer_design(seq, target_tm=55, limit=20), seqs))

for primer in primers:
    print("Forward primer:", primer.forward_primer.seq)
    print("Reverse primer:", primer.reverse_primer.seq)

Forward primer: GTCGACAGTCGCAACAGCAA
Reverse primer: GTCGACGTTTGCCGGCGAGG
Forward primer: GATCGATTACAAACAATTGAGAA
Reverse primer: GCAGATGTTGCTAAAAGTAATT
Forward primer: GTGTCGGTCATCGAATCCAC
Reverse primer: TCAGCGCGCGCCGGACAGAT
Forward primer: ATGGCGCACGGGCTGGGTGG
Reverse primer: TTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGG
Forward primer: ATGTGTTGCGAAGATCACCT
Reverse primer: TCATCCTGTCATGTCCCTGC


In [9]:
enzyme_name = "EcoRI"

enz_seq = AllEnzymes.get(enzyme_name).site.lower()
fwds_primer_list = []
rvss_primer_list = []
for seq in seqs:
    amplicon = primer_design(seq, target_tm=60, limit = 20)
    fwd_primer, rvs_primer = amplicon.primers()
    fwd_primer_enzyme = "tt" + enz_seq + fwd_primer
    rvs_primer_enzyme = "tt" + enz_seq + rvs_primer

    fwds_primer_list.append(fwd_primer_enzyme)
    rvss_primer_list.append(rvs_primer_enzyme)

    print(f"Forward primer: {fwd_primer_enzyme.seq}")
    print(f"Reverse primer: {rvs_primer_enzyme.seq}")

Forward primer: ttgaattcGTCGACAGTCGCAACAGCAA
Reverse primer: ttgaattcGTCGACGTTTGCCGGCGAGG
Forward primer: ttgaattcGATCGATTACAAACAATTGAGAACGA
Reverse primer: ttgaattcGCAGATGTTGCTAAAAGTAATTTCTTAAAC
Forward primer: ttgaattcGTGTCGGTCATCGAATCCAC
Reverse primer: ttgaattcTCAGCGCGCGCCGGACAGAT
Forward primer: ttgaattcATGGCGCACGGGCTGGGTGG
Reverse primer: ttgaattcTTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGG
Forward primer: ttgaattcATGTGTTGCGAAGATCACCT
Reverse primer: ttgaattcTCATCCTGTCATGTCCCTGC


In [10]:
def create_primers(seq, enzyme, target_tm, limit):
    amplicon = primer_design(seq, target_tm=target_tm, limit = target_tm)
    fwd_primer, rvs_primer = amplicon.primers()
    fwd_primer_enzyme = "tt" + enz_seq + fwd_primer
    rvs_primer_enzyme = "tt" + enz_seq + rvs_primer

    return fwd_primer_enzyme, rvs_primer_enzyme

In [11]:
enz_seq = AllEnzymes.get(enzyme_name).site.lower()
primers2 = list(map(lambda seq: create_primers(seq, enz_seq, target_tm=60, limit = 20), seqs))
for fwd_primer_enzyme, rvs_primer_enzyme in primers2:
    print(f"Forward primer: {fwd_primer_enzyme.seq}")
    print(f"Reverse primer: {rvs_primer_enzyme.seq}")

Forward primer: ttgaattcGTCGACAGTCGCAACAGCAATCGAGGGGTGTTGATCAACCTTGGCCAGTTGCCCTTCGTC
Reverse primer: ttgaattcGTCGACGTTTGCCGGCGAGGGCATCGTCTTTCCTCGATTTGAAATTGGGGCCACCGGTGT
Forward primer: ttgaattcGATCGATTACAAACAATTGAGAACGAAAAAGAACAGAGTAAGAATAACGCAGACAAAGCT
Reverse primer: ttgaattcGCAGATGTTGCTAAAAGTAATTTCTTAAACATGAAATCAATCCCTTTCTTTTAAATTATTTACATGTAAATGATAACATTATGTTCAGACTAGAAAAATAATACTTTTTACTCAAATATTAGAATAATTTTGAACTATCTGTAACAAAATAAACATAAGTATCTCCTTG
Forward primer: ttgaattcGTGTCGGTCATCGAATCCACTGACCGAACGAGGCAAGCCGACATGACGCGACACGACATC
Reverse primer: ttgaattcTCAGCGCGCGCCGGACAGATGCAGCGTGAACCACTCCACCGCCACCCGGCTCGTTTCGTC
Forward primer: ttgaattcATGGCGCACGGGCTGGGTGGAATCAAGGAAATGCGGCTCGATGCCTATGCGCAACGCTTT
Reverse primer: ttgaattcTTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGGAACGTGACGGTGCAGAAATGCCAGCTGGTCGGCGACCAC
Forward primer: ttgaattcATGTGTTGCGAAGATCACCTGGTCCCCCCACAAATCCAAAAAATGATTGCGGCCTACCAA
Reverse primer: ttgaattcTCATCCTGTCATGTCCCTGCGCGTTGCTATCGACCGATACTTGTAACGTGCTTGGTCGAG


In [12]:
columns = ["Sequence", "Forward Primer", "Reverse Primer"]
# seq_col = list(itertools.chain.from_iterable(itertools.repeat(x, 2) for x in seqs_id))
# seq_col
df = pd.DataFrame({
    "Sequence": seqs_id,
    "Forward Primer": fwds_primer_list,
    "Reverse Primer": rvss_primer_list
})

df.head()

,Sequence,Forward Primer,Reverse Primer
0,M61151.1,"(t, t, g, a, a, t, t, c, G, T, C, G, A, C, A, ...","(t, t, g, a, a, t, t, c, G, T, C, G, A, C, G, ..."
1,X51424.1,"(t, t, g, a, a, t, t, c, G, A, T, C, G, A, T, ...","(t, t, g, a, a, t, t, c, G, C, A, G, A, T, G, ..."
2,PV006683.1,"(t, t, g, a, a, t, t, c, G, T, G, T, C, G, G, ...","(t, t, g, a, a, t, t, c, T, C, A, G, C, G, C, ..."
3,PV006682.1,"(t, t, g, a, a, t, t, c, A, T, G, G, C, G, C, ...","(t, t, g, a, a, t, t, c, T, T, A, T, A, A, G, ..."
4,OR455449.1,"(t, t, g, a, a, t, t, c, A, T, G, T, G, T, T, ...","(t, t, g, a, a, t, t, c, T, C, A, T, C, C, T, ..."


In [13]:
if not os.path.isdir("results"):
    os.mkdir("results")

df.to_csv("./results/gene_primers.csv", index=False, encoding="utf-8")